# Lesson 26: multilayer perceptron demonstration

## Notebook set up
### Imports

In [ ]:
# Third party imports
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

## 1. Data preparation

### 1.1. Load California housing data

In [ ]:
housing_df = pd.read_csv('https://gperdrizet.github.io/FSA_devops/assets/data/unit2/california_housing.csv')
housing_df.head()

In [ ]:
housing_df.info()

In [ ]:
label = 'MedHouseVal'
features = ['MedInc','HouseAge','AveRooms','AveBedrms','Population','AveOccup','Latitude','Longitude']

### 1.2. Train test split

In [ ]:
training_df, testing_df = train_test_split(housing_df, random_state=42)

### 1.3. Standard scale

#### Features

In [ ]:
feature_scaler = StandardScaler()
feature_scaler.fit(training_df[features])

training_df[features] = feature_scaler.transform(training_df[features])
testing_df[features] = feature_scaler.transform(testing_df[features])

#### Label

In [ ]:
label_scaler = StandardScaler()
label_scaler.fit(training_df[label].to_frame())

training_df[label] = label_scaler.transform(training_df[label].to_frame())
testing_df[label] = label_scaler.transform(testing_df[label].to_frame())

### 1.4. Handle outliers

In [ ]:
for feature in features:

    q1 = training_df[feature].quantile(0.25)
    q3 = training_df[feature].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    training_df[feature] = training_df[feature].clip(lower=lower_bound, upper=upper_bound)
    testing_df[feature] = testing_df[feature].clip(lower=lower_bound, upper=upper_bound)

## 2. Linear regression model

### 2.1. Fit

In [ ]:
linear_model = LinearRegression(n_jobs=-1)
fit_result = linear_model.fit(training_df[features], training_df[label])

### 2.2. Test set evaluation

In [ ]:
linear_predictions = linear_model.predict(testing_df[features])
linear_rsquared = linear_model.score(testing_df[features], testing_df[label])
print(f'Linear Regression R² on test set: {linear_rsquared:.4f}')

### 2.3. Performance analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,4))

axes[0].set_title('Linear regression predictions')
axes[0].scatter(
    testing_df[label],linear_predictions, 
    c='black', s=0.5, alpha=0.5
)
axes[0].plot(
    [testing_df[label].min(), testing_df[label].max()],
    [testing_df[label].min(), testing_df[label].max()], 
    color='red', linestyle='--'
)

axes[0].set_xlabel('True values (standardized)')
axes[0].set_ylabel('Predicted values (standardized)')

axes[1].set_title('Residuals vs predicted values')
axes[1].scatter(
    linear_predictions, testing_df[label] - linear_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Predicted values (standardized)')
axes[1].set_ylabel('Residuals (standardized)')

plt.tight_layout()
plt.show()

## 3. Multilayer perceptron (MLP) model

### 3.1. Single epoch training function

In [ ]:
def train(model: MLPRegressor, df: pd.DataFrame, training_history: dict) -> tuple[MLPRegressor, dict]:
    '''Trains sklearn MLP regression model on given dataframe using validation split.
    returns the updated model and training history dictionary.'''

    df, val_df = train_test_split(df, random_state=315)
    model.partial_fit(df[features], df[label])

    training_history['training_rsquared'].append(model.score(df[features], df[label]))
    training_history['validation_rsquared'].append(model.score(val_df[features], val_df[label]))

    return model, training_history

### 3.2. Model training

In [ ]:
epochs = 10

training_history = {
    'training_rsquared': [],
    'validation_rsquared': []
}

mlp_model = MLPRegressor(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    learning_rate_init=0.001,
    warm_start=True,
    random_state=315
)

for epoch in range(epochs):
    mlp_model, training_history = train(mlp_model, training_df, training_history)

    print(
        f'Epoch {epoch+1}/{epochs} - ' +
        f'training R²: {training_history["training_rsquared"][-1]:.4f} - ' +
        f'validation R²: {training_history["validation_rsquared"][-1]:.4f}'
    )

### 3.3. Learning curves

In [ ]:
plt.plot(
    range(1, epochs + 1), training_history['training_rsquared'],
    label='Training R²'
)
plt.plot(
    range(1, epochs + 1), training_history['validation_rsquared'],
    label='Validation R²'
)
plt.title('Learning curves')
plt.xlabel('Epochs')
plt.ylabel('R²')
plt.legend()
plt.show()

### 3.4. Test set evaluation

In [ ]:
mlp_predictions = mlp_model.predict(testing_df[features])
mlp_rsquared = mlp_model.score(testing_df[features], testing_df[label])
print(f'MLP R² on test set: {mlp_rsquared:.4f}')

### 3.5. Performance analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,4))

axes[0].set_title('MLP predictions')
axes[0].scatter(
    testing_df[label], mlp_predictions, 
    c='black', s=0.5, alpha=0.5
)
axes[0].plot(
    [testing_df[label].min(), testing_df[label].max()],
    [testing_df[label].min(), testing_df[label].max()], 
    color='red', linestyle='--'
)

axes[0].set_xlabel('True values (standardized)')
axes[0].set_ylabel('Predicted values (standardized)')

axes[1].set_title('Residuals vs predicted values')
axes[1].scatter(
    mlp_predictions, testing_df[label] - mlp_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Predicted values (standardized)')
axes[1].set_ylabel('Residuals (standardized)')

plt.tight_layout()
plt.show()

## 4. Model comparison

In [ ]:
print(f'Linear Regression R² on test set: {linear_rsquared:.4f}')
print(f'MLP R² on test set: {mlp_rsquared:.4f}')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8))

axes[0, 0].set_title('Linear regression predictions')
axes[0, 0].scatter(
    testing_df[label], linear_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[0, 0].plot(
    [testing_df[label].min(), testing_df[label].max()],
    [testing_df[label].min(), testing_df[label].max()],
    color='red', linestyle='--'
)
axes[0, 0].set_xlabel('True values (standardized)')
axes[0, 0].set_ylabel('Predicted values (standardized)')

axes[0, 1].set_title('Linear regression residuals')
axes[0, 1].scatter(
    linear_predictions, testing_df[label] - linear_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[0, 1].axhline(0, color='red', linestyle='--')
axes[0, 1].set_xlabel('Predicted values (standardized)')
axes[0, 1].set_ylabel('Residuals (standardized)')

axes[1, 0].set_title('MLP predictions')
axes[1, 0].scatter(
    testing_df[label], mlp_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[1, 0].plot(
    [testing_df[label].min(), testing_df[label].max()],
    [testing_df[label].min(), testing_df[label].max()],
    color='red', linestyle='--'
)
axes[1, 0].set_xlabel('True values (standardized)')
axes[1, 0].set_ylabel('Predicted values (standardized)')

axes[1, 1].set_title('MLP residuals')
axes[1, 1].scatter(
    mlp_predictions, testing_df[label] - mlp_predictions,
    c='black', s=0.5, alpha=0.5
)
axes[1, 1].axhline(0, color='red', linestyle='--')
axes[1, 1].set_xlabel('Predicted values (standardized)')
axes[1, 1].set_ylabel('Residuals (standardized)')

plt.tight_layout()
plt.show()